In [2]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

# external CSS stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


###########################
# Data Manipulation / Model
###########################

# Database connection parameters
username = "aacuser", 
password = "kxMi8CPhAJQj"
collection = "animals"
port = 46319
db = "AAC"
host = "localhost"
shelter = AnimalShelter("localhost", 46319, "AAC", "animals", "aacuser", "kxMi8CPhAJQj")

# class read method must support return of cursor object and accept projection json input
data = shelter.read({})

# Names to show for each column
cols = dict({'animal_id':'ID', 'animal_type':'Type', 'name':'Name', 'breed':'Breed', 'color': 'Color', 'date_of_birth': 'DOB', 'outcome_type':'Outcome', 'age_upon_outcome': 'Age', 'location_long': 'Longitude', 'location_lat': 'Latitude'})
df = pd.DataFrame.from_records(data, columns=['animal_id', 'animal_type', 'name', 'breed', 'color', 'date_of_birth', 'outcome_type', 'age_upon_outcome', 'location_long', 'location_lat'])
initial_active_cell = {"row": 0, "column": 0, "column_id": "animal_id", "row_id": 0}

# Reference to the external stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#########################
# Dashboard Layout / View
#########################

# Instantiation of app variable that references an external stylesheet
app = JupyterDash('Module 5', external_stylesheets=external_stylesheets)

# Rules to apply conditional styles 
# to table
style_data_conditional = [
    {
        'if': {'row_index': 'odd'}, 
        'backgroundColor':'#E8F8FF'
    },
    
]

# Application layout to show all elements it html format
app.layout = html.Div(
    [
        html.Div(
            [
                html.H3("Eric Wallace SNHU CS-340 Module 5 Assignment", style={"textAlign":"center", 'color': '#176EA8'}),
                dash_table.DataTable(
                    id="table",
                    columns=[{"name": cols[c], "id": c} for c in df.columns],
                    data=df.to_dict("records"),
                    page_size=10,
                    sort_action="native",
                    hidden_columns=['location_long', 'location_lat', 'animal_id'],
                    
                    active_cell=initial_active_cell,
                    style_cell = {"fontFamily": "Roboto", "size": 9, 'textAlign': 'center'},
                    style_header={'backgroundColor': '#176EA8', 'color': 'white', 'padding': '5px', 'textAlign': 'center'},
                    style_data_conditional=style_data_conditional,
                    css=[
                        {'selector':'.dash-spreadsheet-menu-item', 'rule': 'display:none'}

                    ]
                ),
            ],
            style={"margin": 50, },
            className="seven columns"
        ),
        html.Div(id="output-map", className="four columns"),
    ],
    className="row"
)

#############################################
# Interaction Between Components / Controller
#############################################
# Callback takes in the active cell and then
# outputs to the conditional
@app.callback(
    Output('table', 'style_data_conditional'),
    [Input('table', 'active_cell')],
)

# Function takes in the active cell
# and then based on the selection it
# highlights the row in which the cell
# is selected
def update_styles(active_cell):
    if active_cell is None:
        return no_update
    row = active_cell['row']
    animal_id = df.at[row, 'animal_id']
    return [{
        'if': { 'row_index' : row},
        'background_color': "#ffe091",
        'border': '1px solid #176EA8',
    }]

# Callback takes in the active cell
# and outputs the map to the output-map
@app.callback(
    Output('output-map', "children"),
    [Input("table", 'active_cell')]
)

# Function that is responsible for updating the map
# this is accomplished by looking up the active cell
# and returning the location of that row of data
def update_map(active_cell):
    if active_cell is None:
        return no_update
    row = active_cell['row']
    long = df.at[row, 'location_long']
    lat = df.at[row, 'location_lat']
    name = df.at[row, 'name'],
    animal_type = df.at[row, 'animal_type']
    
    return [
        dl.Map(style={'width': '400px', 'height': '400px', 'margin-top': '75px'}, center=[lat,long], zoom=9, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat,long], children=[
                dl.Tooltip(animal_type),
                dl.Popup([
                    html.H6(name)
                ])
            ])
        ])
    ]


if __name__ == '__main__':
    app.run_server(mode="external")

Connection successful
Dash app running on http://127.0.0.1:8050/
